# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
# cities_data_file = "../output_data/cities.csv"
cities_data_file = os.path.join('..','output_data','cities.csv')
cities_df = pd.read_csv(cities_data_file)
# print(cities_data_file)
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.1927,168.8643,52.48,85,83,1.77,NZ,1627446024
1,1,Busselton,-33.6500,115.3333,60.04,69,54,19.71,AU,1627446406
2,2,Tautira,-17.7333,-149.1500,78.82,70,79,7.87,PF,1627446406
3,3,Okhotsk,59.3833,143.3000,62.67,79,57,13.09,RU,1627446407
4,4,Punta Alta,-38.8759,-62.0736,43.43,76,91,19.15,AR,1627446352


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [60]:
gmaps.configure(api_key=g_key)

In [86]:
# figure_layout = {
#     'width': '600px',
#     'height': '400px',
#     'border': '1px solid black',
#     'padding': '2px'
# }
locations = cities_df[['Lat', 'Lng']]
weights = cities_df['Humidity']
# fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights,max_intensity = 100,point_radius=5,dissipating=False))
fig


# fig = gmaps.figure()
# heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False)
# heat_layer.max_intensity = 100
# heat_layer.point_radius = 5
# fig.add_layer(heat_layer)
# fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [87]:
ideal_df = cities_df[(cities_df['Max Temp'] >= 70) & (cities_df['Max Temp'] <= 80)]
ideal_df = ideal_df[ideal_df['Wind Speed'] < 10]
ideal_df = ideal_df[ideal_df['Cloudiness'] == 0]
hotel_df = ideal_df
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
88,88,Vila Velha,-20.3297,-40.2925,71.55,83,0,6.91,BR,1627446426
95,95,Uglovoye,44.8215,33.6045,75.40,64,0,5.86,UA,1627446427
178,178,Stara Zagora,42.4328,25.6419,70.43,45,0,2.13,BG,1627446447
224,224,Tataouine,32.9297,10.4518,76.08,69,0,6.51,TN,1627446459
243,243,Monte Alegre,-2.0078,-54.0692,75.24,75,0,3.18,BR,1627446464
266,266,Marsá Maţrūḩ,31.3525,27.2453,79.00,78,0,4.61,EG,1627446469
294,294,Nyzhni Sirohozy,46.8484,34.3804,73.58,64,0,7.76,UA,1627446476
303,303,Jalu,29.0331,21.5482,78.96,46,0,5.68,LY,1627446479
304,304,Semīrom,31.4167,51.5667,70.29,27,0,5.01,IR,1627446479
334,334,Bosilovo,41.4406,22.7278,74.43,45,0,3.00,MK,1627446486


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [88]:
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
88,88,Vila Velha,-20.3297,-40.2925,71.55,83,0,6.91,BR,1627446426,
95,95,Uglovoye,44.8215,33.6045,75.40,64,0,5.86,UA,1627446427,
178,178,Stara Zagora,42.4328,25.6419,70.43,45,0,2.13,BG,1627446447,
224,224,Tataouine,32.9297,10.4518,76.08,69,0,6.51,TN,1627446459,
243,243,Monte Alegre,-2.0078,-54.0692,75.24,75,0,3.18,BR,1627446464,
266,266,Marsá Maţrūḩ,31.3525,27.2453,79.00,78,0,4.61,EG,1627446469,
294,294,Nyzhni Sirohozy,46.8484,34.3804,73.58,64,0,7.76,UA,1627446476,
303,303,Jalu,29.0331,21.5482,78.96,46,0,5.68,LY,1627446479,
304,304,Semīrom,31.4167,51.5667,70.29,27,0,5.01,IR,1627446479,
334,334,Bosilovo,41.4406,22.7278,74.43,45,0,3.00,MK,1627446486,


In [90]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
88,88,Vila Velha,-20.3297,-40.2925,71.55,83,0,6.91,BR,1627446426,Hotel Senac Ilha do Boi
95,95,Uglovoye,44.8215,33.6045,75.40,64,0,5.86,UA,1627446427,Our Sail
178,178,Stara Zagora,42.4328,25.6419,70.43,45,0,2.13,BG,1627446447,Park Hotel Stara Zagora
224,224,Tataouine,32.9297,10.4518,76.08,69,0,6.51,TN,1627446459,NaN
243,243,Monte Alegre,-2.0078,-54.0692,75.24,75,0,3.18,BR,1627446464,Hotel Shekinah
266,266,Marsá Maţrūḩ,31.3525,27.2453,79.00,78,0,4.61,EG,1627446469,Arous El Bahr Hotel
294,294,Nyzhni Sirohozy,46.8484,34.3804,73.58,64,0,7.76,UA,1627446476,Hotelʹ Stepova
303,303,Jalu,29.0331,21.5482,78.96,46,0,5.68,LY,1627446479,إستراحة صلاح الصادق
304,304,Semīrom,31.4167,51.5667,70.29,27,0,5.01,IR,1627446479,(سوئیت اجاره)(ابراهیم دلداده)
334,334,Bosilovo,41.4406,22.7278,74.43,45,0,3.00,MK,1627446486,NaN


In [91]:
hotel_df.to_csv('../output_data/hotels.csv')

In [92]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [93]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))